In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import networkx
import Graph

g = Graph.buildGraph()
# mulGraph = Graph.buildGraph()

Connection pairs:
[
    (7529, 3491, 489.50118772441493),
    (7483, 6868, 128.34041442898507),
    (7487, 7090, 100.0674332727893),
    (7492, 6907, 44.872694234504756),
    (7497, 6861, 45.77161729321111),
    (7498, 6863, 15.894429251450745),
    (7504, 672, 71.5994991372535),
    (7511, 234, 11.285070412497834),
    (7516, 4166, 19.355601312460404),
    (7685, 7508, 26.251222182548965),
    (7487, 7485, 34.69950831729087),
    (7498, 7483, 133.01878867990203),
    (7499, 7489, 161.861689287105),
    (7683, 7518, 839.9571389347612)
]

7529 3491 time = 305.9382423277593 dis = 489.50118772441493 routeVar = Walk

7483 6868 time = 80.21275901811566 dis = 128.34041442898507 routeVar = Walk

7487 7090 time = 62.54214579549331 dis = 100.0674332727893 routeVar = Walk

7492 6907 time = 28.04543389656547 dis = 44.872694234504756 routeVar = Walk

...

7487 7485 time = 21.68719269830679 dis = 34.69950831729087 routeVar = Walk

7498 7483 time = 83.13674292493876 dis = 133.01878867990203 routeVar = Walk

7499 7489 time = 101.16355580444062 dis = 161.861689287105 routeVar = Walk

7683 7518 time = 524.9732118342257 dis = 839.9571389347612 routeVar = Walk

Number of subgraphs: 1

In [ ]:
ug = g.to_undirected()
print(networkx.shortest_path(ug, 35, 7510))
# print(networkx.dijkstra_path(g, 35, 234))
print(networkx.dijkstra_path(g, 7511, 234))

[35, 377, 2568, 2696, 235, 233, 234, 7511, 7515, 7510]
[7511, 234]


In [ ]:
# Return connection pairs
from math import sqrt
from rtree import index
from rich import print

def connectGraph(subGraphs: list[networkx.MultiDiGraph]):
    connectionPair = []
    for i in range(len(subGraphs)):
        for j in range(i + 1, len(subGraphs)):
            idx = index.Index()
            
            nodeCoordI = networkx.get_node_attributes(subGraphs[i], 'coord')
            
            iNodes = list(subGraphs[i].nodes)
            jNodes = list(subGraphs[j].nodes)
            # print(len(list(subGraphs[i].nodes)))
            # print(subGraphs[i].nodes[7278]['coord'])
            # print(subGraphs[i].has_node(7276))
            
            for node in iNodes:
                idx.insert(node, (nodeCoordI[node][0], nodeCoordI[node][1], nodeCoordI[node][0], nodeCoordI[node][1]))
                
            # print(jNodes)
            minDisSoFar = -1
            connection1 = -1
            connection2 = -1
            for node in jNodes:
                # print(subGraphs[j].nodes[7529]['coord'])
                x, y = subGraphs[j].nodes[node]['coord']
                nearestId = list(idx.nearest((x, y, x, y), 1))[0]
                x1, y1 = nodeCoordI[nearestId]
                
                dis = sqrt((x - x1)**2 + (y - y1)**2)
                if minDisSoFar == -1 or minDisSoFar > dis:
                    minDisSoFar = dis
                    connection1 = node
                    connection2 = nearestId
                    
            
            if connection1 == 1:
                print("No points found")
                return
            
            # Only connect two stops that are less than 1km apart
            if minDisSoFar < 1000:
                connectionPair.append((connection1, connection2))
                
    return connectionPair
    

connectGraph(mulGraph)

In [ ]:
# Create edges
def createPath(graph: networkx.MultiDiGraph, stopIdNode: list[str], stopPoints: list[tuple], pathPoints: list[tuple], runningTime, routeVarId):
    pathIndex = -1
    totalLength = LineString(pathPoints).length
    for i in range(len(stopIdNode)):
        if pathIndex == -1:
            pathIndex += 1
            continue
        
        closestIndex = findNextClosest(pathIndex, stopPoints[i], pathPoints)
        line = LineString(pathPoints[pathIndex : closestIndex + 1])
        pathIndex = closestIndex
        # print(stopIdNode[i - 1], stopIdNode[i], line.length)
        graph.add_edge(stopIdNode[i - 1], stopIdNode[i], time = runningTime / totalLength * line.length, dis = line.length, routeVar = routeVarId)
        networkx.set_node_attributes(graph, {i - 1: {"coord": stopPoints[i - 1]}})
        networkx.set_node_attributes(graph, {i: {"coord": stopPoints[i]}})
        
    return graph

In [ ]:
# Get nodes attributes
from rich import print
timeLabel = networkx.get_edge_attributes(g, 'time')
print(g.nodes[35]['coord'])
print(g[7526][7529][0]['time'])

In [ ]:
# List of subgraphs
from networkx import connected_components


ug = g.to_undirected()
# k = g.subgraph(list(connected_components(ug))[3])

subGraphs = [g.subgraph(com) for com in list(component for component in list(connected_components(ug)))]
print(len(subGraphs))
# print(subGraphs[0])
# [g.subgraph(component for component in list(connected_components(ug)) )]
# gs = [sub for sub in ]
# print(k)
# print(k.nodes)

# networkx.draw_spring(k, with_labels = True)



1


In [ ]:
# Correct Dijkstra
from rich import print
def Dijkstra1(g: networkx.MultiDiGraph, index = 0):
    minHeap = [[0, list(g.nodes)[index], list(g.nodes)[index]]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in list(g.nodes):
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
            
    return shortestPath

shortest = Dijkstra1(g, 0)
print(shortest) 

In [109]:
import heapq
import rich

def traversePath(paths: dict, u, v):
    if v not in paths or u not in paths:
        return []
    
    path = []
    weight = 0
    while u != v:
        path.insert(0, v)
        v = paths[v][0]
        
    path.insert(0, u)
    return path
        

def dijkstraLocalSearch(g: networkx.MultiDiGraph, startingNode, v, endings: list, maxTotal = 30):
    endingNodes = endings.copy()
    minHeap = [[0, startingNode, startingNode]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        if cur in endingNodes:
            endingNodes.remove(cur)
        
        if len(endingNodes) == 0 or weight > maxTotal:
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath and u != v:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    # for i in list(g.nodes):
    #     if i not in shortestPath:
    #         shortestPath[i] = (-1, -1) 
            
    # pureShortestPath = {}
    # for i in endings:
    #     if i in shortestPath:
    #         pureShortestPath[i] = shortestPath[i]
    return shortestPath
    
def findShortcuts(g: networkx.MultiDiGraph, x):
    pre = list(g.predecessors(x))
    succ = list(g.successors(x))
    # print(pre)
    print(succ)
    shorts = []
    pth = {}
    for u in pre:
        pth = dijkstraLocalSearch(g, u, x, succ)
        # rich.print()
        for v in succ:
            if x not in traversePath(pth, u, v):
                shorts.append((u, v))
        break
    
    return shorts

findShortcuts(g, 35)

[7276, 28, 1297, 86, 1451, 89, 7265]


[(7275, 7276),
 (7275, 28),
 (7275, 1297),
 (7275, 86),
 (7275, 1451),
 (7275, 89),
 (7275, 7265)]

In [59]:
import heapq

def DijkstraFrom(g: networkx.MultiDiGraph, start, allOutgoingNodes: dict):
    allOutgoingNodes = allOutgoingNodes.copy()
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        if cur in allOutgoingNodes:
            allOutgoingNodes.pop(cur)
            
        if len(allOutgoingNodes) == 0:
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in g.nodes:
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
    
    return shortestPath
    
DijkstraFrom(g, 7526, [])

{7526: (7526, 0),
 35: (-1, -1),
 7276: (-1, -1),
 7277: (-1, -1),
 7278: (-1, -1),
 7265: (-1, -1),
 7266: (-1, -1),
 7693: (-1, -1),
 1256: (-1, -1),
 7588: (-1, -1),
 32: (-1, -1),
 31: (-1, -1),
 34: (-1, -1),
 42: (-1, -1),
 44: (-1, -1),
 39: (-1, -1),
 41: (-1, -1),
 43: (-1, -1),
 46: (-1, -1),
 47: (-1, -1),
 45: (-1, -1),
 49: (-1, -1),
 48: (-1, -1),
 50: (-1, -1),
 51: (-1, -1),
 52: (-1, -1),
 53: (-1, -1),
 54: (-1, -1),
 55: (-1, -1),
 56: (-1, -1),
 57: (-1, -1),
 60: (-1, -1),
 58: (-1, -1),
 62: (-1, -1),
 67: (-1, -1),
 63: (-1, -1),
 72: (-1, -1),
 64: (-1, -1),
 4754: (-1, -1),
 73: (-1, -1),
 213: (-1, -1),
 68: (-1, -1),
 74: (-1, -1),
 69: (-1, -1),
 70: (-1, -1),
 71: (-1, -1),
 77: (-1, -1),
 3204: (-1, -1),
 3203: (-1, -1),
 3206: (-1, -1),
 3259: (-1, -1),
 3207: (-1, -1),
 3210: (-1, -1),
 75: (-1, -1),
 3255: (-1, -1),
 3208: (-1, -1),
 3257: (-1, -1),
 3205: (-1, -1),
 3261: (-1, -1),
 3260: (-1, -1),
 3262: (-1, -1),
 3263: (-1, -1),
 204: (-1, -1),
 205

In [9]:
print(len(list(networkx.neighbors(g, 1197))))

2

In [60]:
from rich.progress import track
import time
import json

def nodeNet(g: networkx.MultiDiGraph, node):
    firstTime = True
    shortcuts = 0
    shortcutPairs = []
    pre = g.pred[node]
    succ = g.succ[node]
    for u in pre:
        start = time.time()
        shortestPath = DijkstraFrom(g, u, succ)
        if time.time() - start > .5:
            if firstTime:
                print("----------------")
                firstTime = False
            print(f"{len(pre) = }")
            print(f"{len(succ) = }")
            print(u, time.time() - start)
            
        for w in succ:
            if networkx.is_path(g, [u, w]):
                continue
            totalTime = pre[u][0]['time'] + succ[w][0]['time']
            if totalTime == shortestPath[w][1]:
                shortcuts += 1
                shortcutPairs.append((u, w))
                
    # print(f"{shortcuts = }, {len(pre) = }, {len(succ) = }")
    return shortcuts - (len(pre) + len(succ)), shortcutPairs

def contract(g: networkx.MultiDiGraph, x, contractionOrdder):
    start = time.time()
    stuff, edges = nodeNet(g, x)
    # print(edges)
    if len(edges) > 1000:
        print(len(edges))
    if (time.time() - start > 1):
        print("Node Net:", time.time() - start)
    startE = time.time()
    for edge in edges:
        # print(g.get_edge_data(edge[0], x))
        # if (edge[0] == 35 and edge[1] == 7483):
        #     print("Found it")
        uPath = g.get_edge_data(edge[0], x)[0].get('path', [])
        uData = g.get_edge_data(edge[0], x)[0]
        wPath = g.get_edge_data(x, edge[1])[0].get('path', [])
        wData = g.get_edge_data(x, edge[1])[0]
        # print(f"{uPath = }, {wPath = }")
        # print(f"{uData = }, {wData = }")
        g.add_edge(edge[0], edge[1], time = uData['time'] + wData['time'], dis = uData['dis'] + wData['dis'], path = uData.get('path', []) + [x] + wData.get('path', []))
    if time.time() - startE > 1:
        print("Edges:", time.time() - startE)

def contractionPhrase(d: networkx.MultiDiGraph):
    g: networkx.MultiDiGraph = d.copy()
    edgeDifference = {}
    nodeList = list(g.nodes)
    for x in nodeList:
        edgeDifference[x] = nodeNet(g, x)
        
    contractionOrder = 1
    for x in ((nodeList)):
        # print("Contracting", x, end='')
        x = min(edgeDifference, key=edgeDifference.get)
        contractedIncidentNodes = 0
        start = time.time()
        timed = {}
        for pre in g.pred[x]:
            if edgeDifference.get(pre, None) is None:
                contractedIncidentNodes += 1
                continue
            newTuple = (edgeDifference[pre][0] + 1, edgeDifference[pre][1])
            # print(newTuple)
            edgeDifference.pop(pre)
            edgeDifference[pre] = newTuple
        timed['pre'] = time.time() - start
        startPre = time.time()
        # print("pre")
        for succ in g.succ[x]:
            if edgeDifference.get(succ, None) is None: 
                contractedIncidentNodes += 1
                continue
            newTuple = (edgeDifference[succ][0] + 1, edgeDifference[succ][1])
            edgeDifference.pop(succ)
            edgeDifference[succ] = newTuple
        timed['succ'] = time.time() - startPre
        startSucc = time.time()
        # print('succ')
        # if contractedIncidentNodes < 10:
        contract(g, x, contractionOrder)
        timed['con'] = time.time() - startSucc
        
        networkx.set_node_attributes(g, {x: {"order": contractionOrder}})
        contractionOrder += 1
        edgeDifference.pop(x)
        # print("Node", x, len(list(g.neighbors(x))))
        if (time.time() - start > 5):
            print("Finished", x, "in", time.time() - start, timed)
        # print("remove", x)
    
    print(g)
    return g
    
ed = contractionPhrase(g)

Node Net: 1.0555016994476318
Node Net: 1.6599907875061035
Node Net: 1.347472906112671
Node Net: 1.2241151332855225
Node Net: 1.0483098030090332
Node Net: 2.707213878631592
Node Net: 1.525414228439331
Node Net: 1.999441385269165
Node Net: 1.7928552627563477
Node Net: 1.6543991565704346
Node Net: 2.837576150894165
1017
Node Net: 1.0612475872039795
Node Net: 1.443526029586792
Node Net: 3.800062894821167
Node Net: 2.404812812805176
2146
Node Net: 3.9762635231018066
Node Net: 1.4820826053619385
1564
Node Net: 1.178849697113037
1382
Node Net: 2.270646095275879
1050
Node Net: 2.4456465244293213
1223
Node Net: 1.6479003429412842
Node Net: 2.233660936355591
1278
Node Net: 4.887126684188843
1408
4184
Node Net: 7.453658580780029
Finished 725 in 7.466386079788208 {'pre': 0.0, 'succ': 0.0, 'con': 7.466386079788208}
2912
Node Net: 12.60015606880188
Finished 1920 in 12.60715627670288 {'pre': 0.0, 'succ': 0.0, 'con': 12.60715627670288}
Node Net: 6.954506158828735
Finished 347 in 6.95750617980957 {'pre

In [49]:
print(g)

MultiDiGraph with 4397 nodes and 9974 edges


In [32]:
test = ed.copy()
test.nodes[35]

{'coord': (684720.4742496928, 1190800.3385383594), 'order': 4395}

In [109]:
DijkstraFromTo(ed, 35, 27)

2.150786895917856

In [66]:
def queryPhrase(g: networkx.MultiDiGraph, start):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.succ[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[start]['order']:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath
    
def queryPhraseReversed(g: networkx.MultiDiGraph, start):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        # if cur == end:
        #     break
        
        for u in g.pred[cur]:
            # print(g.nodes[u]['order'], g.nodes[start]['order'])
            if u not in shortestPath and g.nodes[u]['order'] > g.nodes[start]['order']:
                heapq.heappush(minHeap, [weight + g[u][cur][0]['time'], u, cur])
    
    # print(shortestPath)
    return shortestPath
    
fromWhere = 35
toWhere = 7483
d1 = queryPhrase(ed, fromWhere)
d2 = queryPhraseReversed(ed, toWhere)

final_dict = {x:d1[x] for x in d1 if x in d2}

# print(d1)
# print(d2)
# print(final_dict)

minSoFar = -1
middle = -1
for key in final_dict:
    if minSoFar == -1 or minSoFar > d1[key][1] + d2[key][1]:
        minSoFar = d1[key][1] + d2[key][1]
        middle = key
        
print(minSoFar, key)
print(DijkstraFromTo(g, fromWhere, toWhere))
# print(ed.nodes[27]['order'])
for i in networkx.dijkstra_path(g, fromWhere, toWhere):
    print(i, ":", ed.nodes[i]['order'])
print(networkx.dijkstra_path(g, fromWhere, toWhere))
# print(networkx.dijkstra_path_length(g, 14, 27, 'time'))
# print(networkx.dijkstra_path_length(ed, 14, 27, 'time'))

-1 8
251.45928234674776
35 : 4395
1451 : 4294
1344 : 1979
1196 : 4064
1194 : 2103
1198 : 734
1197 : 4276
1201 : 736
1199 : 2105
1202 : 3500
1200 : 737
1204 : 4116
835 : 4338
1257 : 34
1260 : 4139
1261 : 769
1258 : 4372
1356 : 766
1357 : 2697
1359 : 816
1361 : 3232
1362 : 818
1358 : 4003
1360 : 817
1366 : 2118
1055 : 3216
1058 : 4201
1061 : 677
1063 : 2089
1060 : 2656
1065 : 678
1062 : 3217
1066 : 679
1064 : 3876
2138 : 681
2137 : 2793
2141 : 1090
2139 : 4080
2144 : 1091
3662 : 3277
3664 : 1627
3663 : 3619
3666 : 1628
3665 : 2371
3667 : 4146
3669 : 1629
3670 : 2373
3671 : 3001
3683 : 196
3681 : 3806
3684 : 1883
3682 : 3365
6867 : 4219
6868 : 1868
7483 : 4241
[35, 1451, 1344, 1196, 1194, 1198, 1197, 1201, 1199, 1202, 1200, 1204, 835, 1257, 1260, 1261, 1258, 1356, 1357, 1359, 1361, 1362, 1358, 1360, 1366, 1055, 1058, 1061, 1063, 1060, 1065, 1062, 1066, 1064, 2138, 2137, 2141, 2139, 2144, 3662, 3664, 3663, 3666, 3665, 3667, 3669, 3670, 3671, 3683, 3681, 3684, 3682, 6867, 6868, 7483]


In [41]:
d1

{35: (35, 0)}

In [82]:
while len(hepa) > 0:
    if (hepa[0][1] == 35 or hepa[0][1] == 7483):
        print("Here")
    print(heapq.heappop(hepa))

[1, 7265]

[196, 3683]

[238, 2114]

[587, 2122]

[593, 857]

...

[4338, 859]

[4386, 1256]

[4392, 27]

Here

[4395, 35]

In [ ]:
print(min(ed, key=ed.get))
ed[min(ed, key=ed.get)][0] += 1

7265

TypeError: 'tuple' object does not support item assignment

In [ ]:
di = networkx.floyd_warshall(g, 'time')

In [ ]:
from rich.progress import track
import time
import time

calTime = {}

def FloydWarshall(g: networkx.MultiDiGraph):
    start = time.time()
    nodeList = list(g.nodes)
    
    adjArray = {}
    adjArray = {u: {v: (-1, -1) for v in nodeList} for u in nodeList}
    print("Empty adjacent dict in:", time.time() - start)
    start = time.time()
    
    for node in nodeList:
        adjArray[node][node] = (0, node)
        
    edgeList = list(g.edges)
    for u, v, trash in edgeList:
         adjArray[u][v] = (g[u][v][0]['time'], u)
    print("Only intermediate pairs in:", time.time() - start)
    print ("passed")
         
        
    # return
    nodeL = -1
    longestSoFar = -1
    for k in reversed(nodeList):
        print("At", k, end="")
        start = time.time()
        for i in nodeList:
            for j in nodeList:
                if i == j:
                    continue
                if k == i or k == j:
                    continue
                
                if adjArray[i][k][0] == -1 or adjArray[k][j][0] == -1:
                    continue
                
                if adjArray[i][j][0] == -1 or adjArray[i][j][0] > adjArray[i][k][0] + adjArray[k][j][0]:
                    adjArray[i][j] = (adjArray[i][k][0] + adjArray[k][j][0], k)
        
        dur = time.time() - start
        calTime[k] = dur
        if longestSoFar < dur:
            longestSoFar = dur
            nodeL = k
        print(" took:", dur, ", longest so far:", nodeL, "in:", longestSoFar)      
    return adjArray

adj = FloydWarshall(g)


Empty adjacent dict in: 1.634474277496338
Only intermediate pairs in: 0.4219071865081787
passed
At 7686 took: 2.0991671085357666 , longest so far: 7686 in: 2.0991671085357666
At 7685 took: 2.162587881088257 , longest so far: 7685 in: 2.162587881088257
At 7684 took: 2.0559885501861572 , longest so far: 7685 in: 2.162587881088257
At 7683 took: 1.4940078258514404 , longest so far: 7685 in: 2.162587881088257
At 7682 took: 1.5058503150939941 , longest so far: 7685 in: 2.162587881088257
At 7517 took: 1.5824236869812012 , longest so far: 7685 in: 2.162587881088257
At 7518 took: 1.5780808925628662 , longest so far: 7685 in: 2.162587881088257
At 7512 took: 1.4809050559997559 , longest so far: 7685 in: 2.162587881088257
At 7516 took: 1.5352087020874023 , longest so far: 7685 in: 2.162587881088257
At 7511 took: 1.4673316478729248 , longest so far: 7685 in: 2.162587881088257
At 7515 took: 1.5284993648529053 , longest so far: 7685 in: 2.162587881088257
At 7510 took: 1.4754865169525146 , longest so 

In [ ]:
calTime['2252']

NameError: name 'calTime' is not defined

In [ ]:
dic = {}
for i in range(2):
    dic[i] = {}
    dic[i][i + 1] = -1
    
print(dic[1][2])

-1


In [ ]:
edgeL = list(g.edges)
print(len(edgeL))
for u, v, i in edgeL:
    if u == 7276 and v == 35:
        print(u, v)
    # print(g[u][v][0]['time'])
    # break
# print(g)
# g[edgeL[0][0]][edgeL[0][1]][0]

9974


In [ ]:
# Dijkstra Old
def Dijkstra(k: networkx.MultiDiGraph, startingIndex = 0):
    unvisited = list(k.nodes)
    visited = [unvisited.pop(startingIndex)]

    if len(list(networkx.neighbors(k, visited[-1]))) == 0:
        # print("No path to any other node found")
        return
        
    shortestPath = {}
    for node in list(k.nodes):
        if node == visited[0]:
            shortestPath[node] = (node, 0)
        else:
            shortestPath[node] = (-1, -1)
            
    # print(shortestPath)

    while len(unvisited) > 0:
        nodeToConsider = visited[-1]
        
                
        # Update shortestPath
        for node in unvisited:
            dis = shortestPath[node][1]
            
            # Skip node that cant be reached directly
            if networkx.is_path(k, [nodeToConsider, node]) == False:
                continue
            
            weighToHere = shortestPath[nodeToConsider][1]
            minWeight = k[nodeToConsider][node][0]['time']
            if dis == -1:
                shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
            elif shortestPath[node][1] > minWeight + weighToHere:
                    shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
        
        nextNodeToConsider = -1
        minWeightSoFar = -1
        for u in unvisited:
            # Skip node that are not updated yet
            if shortestPath[u][1] == -1:
                continue
            
            # print(shortestPath[u][1])
            if minWeightSoFar == -1:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
            elif shortestPath[u][1] < minWeightSoFar:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
        if nextNodeToConsider == -1:
            # print("Unreachable node found")
            return shortestPath
            
        
        # print(shortestPath)
        # print(f"{nextNodeToConsider=}")
        
        unvisited.remove(nextNodeToConsider)
        visited.append(nextNodeToConsider)
        # break

    # print()
    return shortestPath

Dijkstra(g)

In [ ]:
from rich import print
# print(g.subgraph(list(connected_components(ug))[0]))
grph: networkx.MultiDiGraph = g.subgraph(list(connected_components(ug))[0]).copy()
stops = list(grph.nodes)
stops.sort()
print(len(stops))